## Trade Evaluator
Who comes out ahead on deals most often?

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [471]:
player = pd.read_csv('TPlayer.csv')
trans = pd.read_csv('TransactionTable.csv')
trans.columns

Index(['Date', 'Week', 'TransType', 'Player', 'Position', 'Team', 'From', 'To',
       'By', 'TransCosts'],
      dtype='object')

In [472]:
tradecols = ['Date', 'Week', 'TransType', 'Player', 'Position', 'Team', 'From', 'To']
trades = trans.loc[trans.TransType == 'Trade',tradecols]

In [473]:
nameDict = dict({
    "Gronk's New Groove":"Sam",
    'Do it for George':'Matt',
    'Tight Ends Spread Sheets':'f',
    'Triple Crown':'Nolan',
    "Eatin W's":'Graham',
    'ganggreen215':'Kevin',
    'The Wet Market Pangolins':'Jake',
})
trades.loc[:,'From'] = trades.loc[:,'From'].map(nameDict)
trades.loc[:,'To'] = trades.loc[:,'To'].map(nameDict)
trades

,Date,Week,TransType,Player,Position,Team,From,To
208,2020-09-17 14:13:00,2,Trade,Joe Mixon,RB,CIN,Sam,Matt
209,2020-09-17 14:13:00,2,Trade,Stefon Diggs,WR,BUF,Matt,Sam
314,2020-09-24 00:31:00,3,Trade,David Johnson,RB,HOU,Matt,f
315,2020-09-24 00:31:00,3,Trade,Jerick McKinnon,RB,SF,Matt,f
316,2020-09-24 00:31:00,3,Trade,Allen Robinson,WR,CHI,f,Matt
317,2020-09-24 00:31:00,3,Trade,Le'Veon Bell,RB,KC,f,Matt
468,2020-10-02 00:25:00,4,Trade,Mike Evans,WR,TB,Nolan,Graham
469,2020-10-02 00:25:00,4,Trade,Kenyan Drake,RB,ARI,Nolan,Graham
470,2020-10-02 00:25:00,4,Trade,Ezekiel Elliott,RB,DAL,Graham,Nolan
510,2020-10-03 17:21:00,4,Trade,Jarvis Landry,WR,CLE,Graham,Kevin


So to break this down, each trade can be identified by the date and time, and we are evaluating which team earned more points from their traded players after the trade was made. We are strictly concerned about a player's value, regardless of whether or not that player was a starter and earned points.

Things we want to know:
* each player's production before the trade (maybe we see if anyone is good at selling low, or bad cause they buy the hype) 
* player production after the trade
* number of games started before and after
* Maybe we also look into what their other players at similar positions were doing, in order to see if they were trying to patch a weakness


In [474]:
# let's borrow the unique_player and get_stint functions 
def get_unique_str(row):
    if row['Position'] == 'DEF':
        nsplit = row['Player'].split(' ')
        if 'Football Team' in row['Player']:
            pstr = '%s %s %s'%(nsplit[-3],nsplit[-2],nsplit[-1])
        else:
            pstr = '%s %s '%(nsplit[-2],nsplit[-1])
    else:
        nsplit = row['Player'].split(' ')
        name = ''
        for i in range(1,len(nsplit)):
            if i == len(nsplit)-1:
                name += '%s, '%nsplit[i]
            else:
                name += '%s '%nsplit[i]
        name += '%s.'%nsplit[0][0]
        pstr = '%s %s %s'%(name,row['Position'],row['Team'])
    
    return pstr

### Generating the Player Info
Data we need to get from the transaction df
* Names of the players
* the week of the transaction
* the owners involved

The receiving team gets the points for the week that the trade occurs

In [488]:
# prep the trades df
unis = player.unique_str.unique()
for i,row in trades.iterrows():
    pstr = get_unique_str(row)
    if pstr not in unis:
        match = get_close_matches(pstr,unis,cutoff=0.8)
        if len(match) > 0:
            pstr = match[0]
        else: 
            print('couldnt find a match for %s'%pstr)
            continue
    BeforeMask = (player['unique_str']==pstr) & (player['week']<row['Week']) & (player['owner']==row['From'])
    AfterMask = (player['unique_str']==pstr) & (player['week']>=row['Week']) & (player['owner']==row['To'])
    
    trades.loc[i,'Before'] = player.loc[BeforeMask,'points'].sum()
    trades.loc[i,'B_GamesOnTeam'] = BeforeMask.sum()
    trades.loc[i,'B_AvPoints'] = round((trades.loc[i,'Before'] / trades.loc[i,'B_GamesOnTeam']),2)
    trades.loc[i,'B_GamesStarted'] = len(player.loc[BeforeMask & (player['status']=='Active'),:])
    trades.loc[i,'B_%Started'] = len(player.loc[BeforeMask & (player['status']=='Active'),:])/row.Week
    
    trades.loc[i,'After'] = player.loc[AfterMask,'points'].sum()
    trades.loc[i,'A_GamesOnTeam'] = AfterMask.sum()
    trades.loc[i,'A_AvPoints'] = round((trades.loc[i,'After'] / trades.loc[i,'A_GamesOnTeam']),2)
    trades.loc[i,'A_GamesStarted'] = len(player.loc[AfterMask & (player['status']=='Active'),:])
    trades.loc[i,'A_%Started'] = len(player.loc[AfterMask & (player['status']=='Active'),:])/(17-row.Week)
trades

,Date,Week,TransType,Player,Position,Team,From,To,Before,B_GamesOnTeam,B_AvPoints,B_GamesStarted,B_%Started,After,A_GamesOnTeam,A_AvPoints,A_GamesStarted,A_%Started
208,2020-09-17 14:13:00,2,Trade,Joe Mixon,RB,CIN,Sam,Matt,6.1,1.0,6.10,1.0,0.500000,93.50,15.0,6.23,4.0,0.266667
209,2020-09-17 14:13:00,2,Trade,Stefon Diggs,WR,BUF,Matt,Sam,16.6,1.0,16.60,1.0,0.500000,297.40,15.0,19.83,13.0,0.866667
314,2020-09-24 00:31:00,3,Trade,David Johnson,RB,HOU,Matt,f,26.9,2.0,13.45,1.0,0.333333,131.60,14.0,9.40,10.0,0.714286
315,2020-09-24 00:31:00,3,Trade,Jerick McKinnon,RB,SF,Matt,f,35.8,2.0,17.90,0.0,0.000000,112.70,14.0,8.05,11.0,0.785714
316,2020-09-24 00:31:00,3,Trade,Allen Robinson,WR,CHI,f,Matt,18.6,2.0,9.30,2.0,0.666667,238.60,14.0,17.04,12.0,0.857143
317,2020-09-24 00:31:00,3,Trade,Le'Veon Bell,RB,KC,f,Matt,6.6,2.0,3.30,1.0,0.333333,30.30,8.0,3.79,3.0,0.214286
468,2020-10-02 00:25:00,4,Trade,Mike Evans,WR,TB,Nolan,Graham,44.8,3.0,14.93,2.0,0.500000,196.20,13.0,15.09,10.0,0.769231
469,2020-10-02 00:25:00,4,Trade,Kenyan Drake,RB,ARI,Nolan,Graham,34.9,3.0,11.63,3.0,0.750000,149.20,13.0,11.48,8.0,0.615385
470,2020-10-02 00:25:00,4,Trade,Ezekiel Elliott,RB,DAL,Graham,Nolan,67.7,3.0,22.57,3.0,0.750000,140.90,13.0,10.84,12.0,0.923077
510,2020-10-03 17:21:00,4,Trade,Jarvis Landry,WR,CLE,Graham,Kevin,26.3,3.0,8.77,1.0,0.250000,35.08,3.0,11.69,0.0,0.000000


### Generating the GM scores
So what do we have so far? The above dataframe outlines the contribution of each player involved in a trade to the sending team and the receiving team. 
* we would like to know how many points were gained by a GM as a result of a trade (the net improvement or loss as a result of trading the player)
* we would also like to know how many games were started by that player


So here's what I'm trying to capture

* Improve?: the points earned by a  GM after trading away a player i.e. the points they gained from their new player vs the points they lost from trading players away. This relationship is zero sum between the two trading GMs and shows if a GM can predict and project a player's **future** performance. 

* At The Time: the difference in points before the trade occurred. This was the data that GM were working with before the trade happened. So comparing the average points earned by the players traded away vs the points earned by the player(s) received. A GM's net captures a GM's decision making at the time of trade based on a player's **past** performances

* %Started: I'm interested if a GM will trade for players that they will eventually start, because even if you won a trade, if you don't start that player then the trade is in effect a zero

In [490]:
tradelogA = pd.DataFrame(columns = ['Date','GM','Improve?','AtTheTime','%Started'])
tradelogB = pd.DataFrame(columns = ['Date','GM','Improve?','AtTheTime','%Started'])
for i,date in enumerate(set(trades.Date)):
    tradeIso = trades.loc[trades['Date']==date,:]
    tradelogA.loc[i,'Date'] = date
    tradelogB.loc[i,'Date'] = date
    GM1 = tradeIso.iloc[0]['From']
    GM2 = tradeIso.iloc[0]['To']
    tradelogA.loc[i,'GM'] = tradeIso.iloc[0]['From']
    tradelogB.loc[i,'GM'] = tradeIso.iloc[0]['To']
    # this is ugly af
    tradelogA.loc[i,'Improve?'] = (tradeIso.loc[tradeIso['To']==GM1,'A_AvPoints'].sum() - tradeIso.loc[tradeIso['From']==GM1,'A_AvPoints'].sum())
    tradelogA.loc[i,'AtTheTime'] = (tradeIso.loc[tradeIso['To']==GM1,'B_AvPoints'].sum() - tradeIso.loc[tradeIso['From']==GM1,'B_AvPoints'].sum())
    tradelogA.loc[i,'%Started'] = (tradeIso.loc[tradeIso['To']==GM1,'A_GamesStarted'].sum() / ((17-tradeIso.loc[tradeIso['To']==GM1,'Week'].iloc[0])*sum(tradeIso.To==GM1)))
    
    tradelogB.loc[i,'Improve?'] = (tradeIso.loc[tradeIso['To']==GM2,'A_AvPoints'].sum() - tradeIso.loc[tradeIso['From']==GM2,'A_AvPoints'].sum())
    tradelogB.loc[i,'AtTheTime'] = (tradeIso.loc[tradeIso['To']==GM2,'B_AvPoints'].sum() - tradeIso.loc[tradeIso['From']==GM2,'B_AvPoints'].sum())
    tradelogB.loc[i,'%Started'] = (tradeIso.loc[tradeIso['To']==GM2,'A_GamesStarted'].sum() / ((17-tradeIso.loc[tradeIso['To']==GM2,'Week'].iloc[0])*sum(tradeIso.To==GM2)))

GMPerf = pd.concat([tradelogA,tradelogB]).sort_values('Date').reset_index(drop = True)
GMPerf[['Improve?','AtTheTime','%Started']] = GMPerf[['Improve?','AtTheTime','%Started']].astype(float)

In [480]:
def trade_info(df):
    for i,row in df.iterrows():
        tradeIso = trades.loc[trades.Date == row['Date'],:]
        df.loc[i,'Sent'] = tradeIso.loc[tradeIso['From']==row['GM'],'Player'].str.cat(sep = ", ")
        df.loc[i,'Received'] = tradeIso.loc[tradeIso['To']==row['GM'],'Player'].str.cat(sep = ", ")
        df.loc[i,'WeeksOnTeam'] = 17-tradeIso['Week'].iloc[0]

    return(df)

In [481]:
trade_info(GMPerf)

,Date,GM,Improve?,AtTheTime,%Started,Sent,Received,WeeksOnTeam
0,2020-09-17 14:13:00,Sam,13.60,10.50,0.866667,Joe Mixon,Stefon Diggs,15.0
1,2020-09-17 14:13:00,Matt,-13.60,-10.50,0.266667,Stefon Diggs,Joe Mixon,15.0
2,2020-09-24 00:31:00,Matt,3.38,-18.75,0.535714,"David Johnson, Jerick McKinnon","Allen Robinson, Le'Veon Bell",14.0
3,2020-09-24 00:31:00,f,-3.38,18.75,0.750000,"Allen Robinson, Le'Veon Bell","David Johnson, Jerick McKinnon",14.0
4,2020-10-02 00:25:00,Nolan,-15.73,-3.99,0.923077,"Mike Evans, Kenyan Drake",Ezekiel Elliott,13.0
5,2020-10-02 00:25:00,Graham,15.73,3.99,0.692308,Ezekiel Elliott,"Mike Evans, Kenyan Drake",13.0
6,2020-10-03 17:21:00,Graham,-6.87,-9.30,0.692308,"Jarvis Landry, James Conner",Jonathan Taylor,13.0
7,2020-10-03 17:21:00,Kevin,6.87,9.30,0.307692,Jonathan Taylor,"Jarvis Landry, James Conner",13.0
8,2020-10-24 00:39:00,Sam,-1.09,-1.52,0.500000,Terry McLaurin,D'Andre Swift,10.0
9,2020-10-24 00:39:00,Kevin,1.09,1.52,0.800000,D'Andre Swift,Terry McLaurin,10.0


### 5 best trades in terms of value added

In [483]:
Best5 = GMPerf.sort_values('Improve?', ascending = False, key = abs).head(10)
Best5

,Date,GM,Improve?,AtTheTime,%Started,Sent,Received,WeeksOnTeam
4,2020-10-02 00:25:00,Nolan,-15.73,-3.99,0.923077,"Mike Evans, Kenyan Drake",Ezekiel Elliott,13.0
5,2020-10-02 00:25:00,Graham,15.73,3.99,0.692308,Ezekiel Elliott,"Mike Evans, Kenyan Drake",13.0
0,2020-09-17 14:13:00,Sam,13.60,10.50,0.866667,Joe Mixon,Stefon Diggs,15.0
1,2020-09-17 14:13:00,Matt,-13.60,-10.50,0.266667,Stefon Diggs,Joe Mixon,15.0
12,2020-11-29 09:40:00,Jake,9.16,3.88,1.000000,Aaron Jones,Tyreek Hill,5.0
13,2020-11-29 09:40:00,Graham,-9.16,-3.88,1.000000,Tyreek Hill,Aaron Jones,5.0
10,2020-11-01 00:14:00,Matt,8.10,-4.04,1.000000,Clyde Edwards-Helaire,A.J. Brown,9.0
11,2020-11-01 00:14:00,Jake,-8.10,4.04,0.333333,A.J. Brown,Clyde Edwards-Helaire,9.0
6,2020-10-03 17:21:00,Graham,-6.87,-9.30,0.692308,"Jarvis Landry, James Conner",Jonathan Taylor,13.0
7,2020-10-03 17:21:00,Kevin,6.87,9.30,0.307692,Jonathan Taylor,"Jarvis Landry, James Conner",13.0


Thoughts:
* Graham flipping Zeke for Mike Evans and Kenyan Drake was the best overall, however we haven't accounted for the value of the replacement player in Nolan's roster so this one might not be as bad. It was definitely a Zeke sell high though
* Sam was winning the Joe Mixon for Stefon Diggs trade when it happened and it only got better over the rest of the season, both in terms of points earned and %started
* Matt's trade with Jake was a sell high trade, where Edwards-Helarie was producing better at the time, but AJ Brown was more valuable over the rest of the season

### Average GM Performance

In [487]:
GMAvPerf = GMPerf.groupby('GM').mean().sort_values('Improve?', ascending = False)
GMAvPerf

,Improve?,AtTheTime,%Started,WeeksOnTeam
GM,,,,
Sam,6.255000,4.490000,0.683333,12.500000
Kevin,3.980000,5.410000,0.553846,11.500000
Jake,0.530000,3.960000,0.666667,7.000000
Graham,-0.100000,-3.063333,0.794872,10.333333
Matt,-0.706667,-11.096667,0.600794,12.666667
f,-3.380000,18.750000,0.750000,14.000000
Nolan,-15.730000,-3.990000,0.923077,13.000000


Thoughts:
* Sam arguably had the move of the year trading for Diggs and helped mitigate some of his other moves
* Alex had a brutal buy high trade where he bought players that were producing significantly higher than the ones he traded away, but were worse off for the rest of the year

Future thoughts:
* In 3/4 trades involving an RB for a WR straight up, people accepted an average lower points production by the RB for the WR. How do we weight RB value vs WR value?
* We need a way to account for the value of the replacement player in 2:1 or other mismatch trades


In [489]:
trades

,Date,Week,TransType,Player,Position,Team,From,To,Before,B_GamesOnTeam,B_AvPoints,B_GamesStarted,B_%Started,After,A_GamesOnTeam,A_AvPoints,A_GamesStarted,A_%Started
208,2020-09-17 14:13:00,2,Trade,Joe Mixon,RB,CIN,Sam,Matt,6.1,1.0,6.10,1.0,0.500000,93.50,15.0,6.23,4.0,0.266667
209,2020-09-17 14:13:00,2,Trade,Stefon Diggs,WR,BUF,Matt,Sam,16.6,1.0,16.60,1.0,0.500000,297.40,15.0,19.83,13.0,0.866667
314,2020-09-24 00:31:00,3,Trade,David Johnson,RB,HOU,Matt,f,26.9,2.0,13.45,1.0,0.333333,131.60,14.0,9.40,10.0,0.714286
315,2020-09-24 00:31:00,3,Trade,Jerick McKinnon,RB,SF,Matt,f,35.8,2.0,17.90,0.0,0.000000,112.70,14.0,8.05,11.0,0.785714
316,2020-09-24 00:31:00,3,Trade,Allen Robinson,WR,CHI,f,Matt,18.6,2.0,9.30,2.0,0.666667,238.60,14.0,17.04,12.0,0.857143
317,2020-09-24 00:31:00,3,Trade,Le'Veon Bell,RB,KC,f,Matt,6.6,2.0,3.30,1.0,0.333333,30.30,8.0,3.79,3.0,0.214286
468,2020-10-02 00:25:00,4,Trade,Mike Evans,WR,TB,Nolan,Graham,44.8,3.0,14.93,2.0,0.500000,196.20,13.0,15.09,10.0,0.769231
469,2020-10-02 00:25:00,4,Trade,Kenyan Drake,RB,ARI,Nolan,Graham,34.9,3.0,11.63,3.0,0.750000,149.20,13.0,11.48,8.0,0.615385
470,2020-10-02 00:25:00,4,Trade,Ezekiel Elliott,RB,DAL,Graham,Nolan,67.7,3.0,22.57,3.0,0.750000,140.90,13.0,10.84,12.0,0.923077
510,2020-10-03 17:21:00,4,Trade,Jarvis Landry,WR,CLE,Graham,Kevin,26.3,3.0,8.77,1.0,0.250000,35.08,3.0,11.69,0.0,0.000000
